In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf


In [3]:
df = pd.read_csv('SolAtasIMC_tratado.csv')
df.head()

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36400 entries, 0 to 36399
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    36400 non-null  object 
 1   open    36400 non-null  float64
 2   high    36400 non-null  float64
 3   low     36400 non-null  float64
 4   close   36400 non-null  float64
 5   value   36400 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 1.7+ MB
None


# Preprocesado de Datos

In [5]:
tamanio = df.shape[0]

In [6]:
df_train = df.copy().loc[0:int(tamanio*0.7)]
df_train

,date,open,high,low,close,value
0,2020-08-12 03:00:00,3.10,3.35,3.10,3.25,75
1,2020-08-12 04:00:00,3.25,3.25,3.15,3.15,75
2,2020-08-12 05:00:00,3.15,3.30,3.15,3.30,75
3,2020-08-12 06:00:00,3.30,3.30,3.15,3.30,75
4,2020-08-12 07:00:00,3.25,3.25,3.20,3.25,75
...,...,...,...,...,...,...
25476,2023-07-11 21:00:00,22.00,22.05,21.90,22.00,57
25477,2023-07-11 22:00:00,22.00,22.10,21.90,22.00,57
25478,2023-07-11 23:00:00,22.00,22.05,21.75,21.95,57
25479,2023-07-12 00:00:00,21.95,22.10,21.90,22.05,64


In [7]:
df_vali = df.copy().loc[int(tamanio*0.7 + 1):int(tamanio*0.9)]
df_vali

,date,open,high,low,close,value
25481,2023-07-12 02:00:00,22.10,22.30,22.05,22.15,64
25482,2023-07-12 03:00:00,22.15,22.25,22.10,22.10,64
25483,2023-07-12 04:00:00,22.10,22.10,22.00,22.00,64
25484,2023-07-12 05:00:00,22.00,22.00,21.90,21.95,64
25485,2023-07-12 06:00:00,21.95,22.05,21.90,22.00,64
...,...,...,...,...,...,...
32756,2024-05-10 05:00:00,153.65,154.35,152.85,153.95,66
32757,2024-05-10 06:00:00,153.95,154.70,153.45,153.75,66
32758,2024-05-10 07:00:00,153.75,154.10,152.30,153.30,66
32759,2024-05-10 08:00:00,153.30,155.10,153.15,154.95,66


In [8]:
df_test = df.copy().loc[int(tamanio*0.9 + 1):tamanio]
df_test

,date,open,high,low,close,value
32761,2024-05-10 10:00:00,154.35,154.5,153.45,154.10,66
32762,2024-05-10 11:00:00,154.10,154.8,153.25,154.15,66
32763,2024-05-10 12:00:00,154.15,154.3,153.25,154.15,66
32764,2024-05-10 13:00:00,154.15,155.2,153.00,155.05,66
32765,2024-05-10 14:00:00,155.05,155.4,153.10,153.30,66
...,...,...,...,...,...,...
36395,2024-10-08 20:00:00,143.35,143.9,142.35,142.95,49
36396,2024-10-08 21:00:00,142.95,144.1,142.25,143.75,49
36397,2024-10-08 22:00:00,143.75,144.5,143.35,144.50,49
36398,2024-10-08 23:00:00,144.50,144.7,144.05,144.25,49


In [9]:
df_valitest = pd.concat([df_vali, df_test], axis=0)

Numero de horas que se utilizan en la predicción

In [10]:
numhorasconst = 4

# Redes neuronales Densas

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [12]:
print("¿GPU detectada?:", tf.config.list_physical_devices('GPU'))

¿GPU detectada?: []


In [13]:
print("Versión de TensorFlow:", tf.__version__)

Versión de TensorFlow: 2.18.0


In [14]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 3])  
    return np.array(X), np.array(y)

In [15]:
def preparar_datos(df, numhoras):
    data = df[['open', 'high', 'low', 'close', 'value']].values
    X, y = create_sequences(data, numhoras)
    X_aux = []
    for i in X:
        aux = []
        for r in range(0, numhoras):
            for elem in i[r]:
                aux.append(elem)
        X_aux.append(aux)       
    X_aux = np.array(X_aux) 
    return X_aux, y

In [16]:
def evalRedDensa(ytest, y_pred):
    y_pred = y_pred.flatten()
    suma = 0
    n = len(y_pred)
    for i in range(0,n):
        suma = abs(y_pred[i] - ytest[i])/ytest[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [23]:
def opti_redes_densas(epoch_ini, epoch_fin, batch_array, numhoras, X_train, y_train, X_vali, y_vali, X_test, y_test):
    best = 100
    epoch_best = 0
    bacth_best = 0
    best_model = None
    for e in range(epoch_ini, epoch_fin + 1):
        for b in batch_array:
            best_value_of_the25 = 100
            best_model_of_the25 = None
            for i in range(0, 25):#Número de veces que se entrena cada modelo   
                model = Sequential()
                model.add(Dense(64, activation='relu', input_shape=(numhoras*5,)))
                model.add(Dense(64, activation='relu'))
                model.add(Dense(1))
                model.compile(optimizer='adam', loss='mape')
                history = model.fit(X_train, y_train, epochs=e, batch_size=b, validation_data=(X_vali, y_vali), shuffle=False)
                y_pred = model.predict(X_test)
                valor = evalRedDensa(y_test, y_pred)
                if valor < best_value_of_the25:
                    best_value_of_the25 = valor
                    best_model_of_the25 = model
            print("epoch: "+str(e)+", batch_size: "+str(b)+", value: "+str(best_value_of_the25))
            with open('pasosdados.txt', 'w') as archivo:
                archivo.write("epoch: "+str(e)+", batch_size: "+str(b) +"\n")
            if best_value_of_the25 < best:
                best = best_value_of_the25
                epoch_best = e
                bacth_best = b
                cadena_guardado = "ModelosDensosOptiMoreDataIMC/mi_modelo_denso_Opti_e"+str(e)+"_b"+str(b)+"_v"+str(round(best_value_of_the25, 3)+"_nh"+str(numhoras))
                best_model_of_the25.save(cadena_guardado+".h5")
                best_model_of_the25.save(cadena_guardado+".keras")
                best_model = best_model_of_the25
    return epoch_best, bacth_best, valor, best_model 

In [24]:
def opti_rd_h(inih, finh, epoch_ini, epoch_fin, batch_array):
    best = 100
    epoch_best = 0
    bacth_best = 0
    h_best = 0
    best_model = None
    for i in range(inih, finh+1):
        Xtrain, ytrain = preparar_datos(df_train, i)
        Xvali, yvali = preparar_datos(df_vali, i)
        Xtest, ytest = preparar_datos(df_test, i)
        valores = opti_redes_densas(epoch_ini, epoch_fin, batch_array, i, Xtrain, ytrain, Xvali, yvali, Xtest, ytest)
        if valores[2] < best:
            best = valores[2]
            epoch_best = valores[0]
            bacth_best = valores[1]
            h_best = i
            best_model = valores[3]
            cadena_guardado = "ModelosDensosOptiMoreDataIMCBest/mi_modelo_denso_Opti_e"+str(epoch_best)+"_b"+str(bacth_best)+"_h"+str(i)+"_v"+str(round(best, 3)+"_nh"+str(i))
            best_model.save(cadena_guardado+".h5")
            best_model.save(cadena_guardado+".keras")
        with open('pasosdadoshoras.txt', 'w') as archivo:
            archivo.write("horas: "+str(i)+"\n")
    return best, epoch_best, bacth_best, h_best, best_model

In [22]:
data = opti_rd_h(7, 16, 3, 15, [4, 6, 8, 12, 16, 24, 32, 46, 64, 96, 128, 256])

Epoch 1/3


c:\Users\raulg\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6369/6369 ━━━━━━━━━━━━━━━━━━━━ 6s 802us/step - loss: 12.9824 - val_loss: 13.4089
Epoch 2/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 5s 782us/step - loss: 7.6837 - val_loss: 8.6074
Epoch 3/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 5s 844us/step - loss: 5.1047 - val_loss: 8.6145
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step
Epoch 1/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 6s 802us/step - loss: 17.7438 - val_loss: 25.7001
Epoch 2/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 5s 792us/step - loss: 7.7851 - val_loss: 12.7188
Epoch 3/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 5s 800us/step - loss: 5.4078 - val_loss: 7.4076
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step
Epoch 1/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 6s 795us/step - loss: 12.6637 - val_loss: 18.1588
Epoch 2/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 5s 789us/step - loss: 6.6123 - val_loss: 9.5441
Epoch 3/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 5s 783us/step - loss: 5.2128 - val_loss: 6.4979
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step
Epoch 1/3
6369/6369 ━━━━━━━━━━━━━━━━━━━━ 6s 790us/step - loss: 16.6794 - 

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U3')) -> None

# Redes neuronales LSTM

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 3])  
    return np.array(X), np.array(y)

In [ ]:
def preparar_datosLSTM(df, numhoras):
    return create_sequences(df[['open', 'high', 'low', 'close', 'value']].values, numhoras)

In [ ]:
def evalRedLSTM(ytest, y_pred):
    y_pred = y_pred.flatten()
    suma = 0
    n = len(y_pred)
    for i in range(0,n):
        suma = abs(y_pred[i] - ytest[i])/ytest[i] +  suma
    error_medio = suma/n
    emp = error_medio*100 # error medio en porcentaje
    return emp

In [ ]:
def opti_redes_LSTM(epoch_ini, epoch_fin, batch_array, X_trainLSTM, y_trainLSTM, X_valiLSTM, y_valiLSTM, X_testLSTM, y_testLSTM, numhoras):
    best = 100
    epoch_best = 0
    bacth_best = 0
    best_model = None
    for e in range(epoch_ini, epoch_fin + 1):
        for b in batch_array:
            modelLSTM = Sequential()
            modelLSTM.add(LSTM(64, activation='relu', input_shape=(numhoras, 5)))
            modelLSTM.add(Dense(1))
            modelLSTM.compile(optimizer='adam', loss='mape')
            historyLSTM = modelLSTM.fit(X_trainLSTM, y_trainLSTM, epochs=e, batch_size=b, validation_data=(X_valiLSTM, y_valiLSTM), shuffle=False)
            y_pred = modelLSTM.predict(X_testLSTM)
            valor = evalRedLSTM(y_testLSTM, y_pred)
            print("epoch: "+str(e)+", batch_size: "+str(b)+", value: "+str(valor))
            if valor < best:
                best = valor
                epoch_best = e
                bacth_best = b
                best_model = modelLSTM
                cadena_guardado = "ModelosLSTMOptiMoreDataIMC/mi_modelo_LSTMOpti_e"+str(e)+"_b"+str(b)+"_v"+str(round(valor, 3))
                best_model.save(cadena_guardado+".h5")
                best_model.save(cadena_guardado+".keras")
    return epoch_best, bacth_best, valor, best_model

In [ ]:
BATCH_ARRAY = [4, 8, 12, 16, 20, 24, 28, 32, 40, 48, 64, 96, 128, 192, 256]

In [ ]:
def opti_rLSTM_h(inih, finh, epoch_ini, epoch_fin, batch_array):
    best = 100
    epoch_best = 0
    bacth_best = 0
    h_best = 0
    best_model = None
    for i in range(inih, finh+1):
        Xtrain, ytrain = preparar_datosLSTM(df_train, i)
        Xvali, yvali = preparar_datosLSTM(df_vali, i)
        Xtest, ytest = preparar_datosLSTM(df_test, i)
        valores = opti_redes_LSTM(epoch_ini, epoch_fin, batch_array, Xtrain, ytrain, Xvali, yvali, Xtest, ytest, i)
        if valores[2] < best:
            best = valores[2]
            epoch_best = valores[0]
            bacth_best = valores[1]
            h_best = i
            best_model = valores[3]
            cadena_guardado = "ModelosLSTMOptiMoreDataIMCBest/mi_modelo_LSTM_Opti_e"+str(epoch_best)+"_b"+str(bacth_best)+"_h"+str(i)+"_v"+str(round(best, 3))
            best_model.save(cadena_guardado+".h5")
            best_model.save(cadena_guardado+".keras")
    return best, epoch_best, bacth_best, h_best, best_model

In [ ]:
data = opti_rLSTM_h(7, 16, 3, 20, BATCH_ARRAY)